# Imports


In [164]:
import random
import gradio as gr
import copy

cross_prob = 0.65
mut_prob = 0.05

# Flower



In [151]:
class Flower:
    def __init__(self, sizeCenter , colorCenter , colorPetals , colorStem , numPetals): 
        self.sizeCenter = sizeCenter
        self.colorCenter = colorCenter
        self.colorPetals = colorPetals
        self.colorStem = colorStem
        self.numPetals = numPetals
        self.fitness = 1

        self.attribute = [
            self.sizeCenter, 
            *self.colorCenter,
            *self.colorPetals,
            *self.colorStem,
            self.numPetals,
            # self.fitness
        ]

    def set_fitness(self,new_fit):
        self.fitness = new_fit
        
    def display(self):
        print(f"------------- Flower Details: -------------")
        print(f"Center Size: {self.sizeCenter}")
        print(f"Center Color: {self.colorCenter}")
        print(f"Petals Color: {self.colorPetals}")
        print(f"Stem Color: {self.colorStem}")
        print(f"Number of Petals: {self.numPetals}")
        print(f"Fitness value: {self.fitness}")
        print(f"array of Attributes: {self.attribute}")



# Population


In [152]:
# Assume sizeCenter is 8 bits 
# Assume numPetals is 8 bits
# we may change the colorCenter, colorPetals, colorStem to be 24 bits each (8 bits for R, 8 bits for G, 8 bits for B)

def createPopulation():
    population = []  
    for i in range(8):
        sizeCenter = random.randint(10, 30)
        colorCenter = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        colorPetals = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        colorStem = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        numPetals = random.randint(0, 7)
        flower = Flower(sizeCenter, colorCenter, colorPetals, colorStem, numPetals)
        population.append(flower)
    return population



In [ ]:
# test createPopulation
# population = createPopulation()
# population[0].display()
# population[1].display()
# population[2].display()
# population[3].display()
# population[4].display()
# population[5].display()
# population[6].display()
# population[7].display()

In [153]:
def convertDecimalToBinary(n):
    return bin(n).replace("0b", "").zfill(8)

def convertDecimalToBinary3(n):
    return bin(n).replace("0b", "").zfill(3)

def convertDecimalToBinary5(n):
    return bin(n).replace("0b", "").zfill(5)

print(convertDecimalToBinary(7))


00000111


In [154]:
def displayBinaryRepresentation(flower):
    sizeCenterBinary = convertDecimalToBinary5(flower.sizeCenter) # 5 bits to encapsulate 30 ( in the mail 10 to 30)
    colorCenterBinary = ''.join([convertDecimalToBinary(c) for c in flower.colorCenter]) 
    colorPetalsBinary = ''.join([convertDecimalToBinary(c) for c in flower.colorPetals])
    colorStemBinary = ''.join([convertDecimalToBinary(c) for c in flower.colorStem])
    numPetalsBinary = convertDecimalToBinary3(flower.numPetals)
    
    chromosome = sizeCenterBinary + colorCenterBinary + colorPetalsBinary + colorStemBinary + numPetalsBinary
    return chromosome

def Chromosomes(population):
    result = []
    for i, flower in enumerate(population):
        result.append(displayBinaryRepresentation(flower))
    return result


  

In [ ]:
# test displays binary

# for flower in population:
#     flower.display()
#     print("Binary Representation: ", displayBinaryRepresentation(flower))
#     print()


# Chromosomes = Chromosomes(population)
# for i, chromosome in enumerate(Chromosomes):
#     print(f"Flower {i+1} Chromosome: {chromosome}")   

In [155]:
def convertBinaryToDecimal(chromosome):
    decimal = 0
    for i in range(len(chromosome)):
        decimal += int(chromosome[i]) * (2 ** (len(chromosome) - 1 - i))
    return decimal
   
    # return int(chromosome, 2) does the same 


In [62]:
# print(convertBinaryToDecimal("00110001"))

In [156]:
def repair(min_value, max_value,x_value):
    return max(min_value, min(max_value, x_value))

# min 0
# max 1000

# num > 1000 2000
# num < 0  -1000

In [64]:
# Create population  
# Create Flower 
# Selection 
# Crossover
# Mutation
# Fitness Function
# Main 
# GUI 

# Selection


In [157]:
import random

# we can change this slightly to return the reordered list
#  with the probabilities of each flower for visualization

def roulette_selection(population):
    
    remaining = population.copy()
    new_population = []

    for _ in range(len(population)):
        total_fitness = sum(f.fitness for f in remaining)

        cumaltive = []
        acc = 0 
        for f in remaining:
            acc += f.fitness / total_fitness
            cumaltive.append(acc)
        
        r = random.random()

        for i, f in enumerate(remaining):
            if r <= cumaltive[i]:
                new_population.append(f)
                remaining.pop(i)
                break

    return new_population

In [ ]:
# # example usage

# st_population = createPopulation()

# for f in (st_population):
#     f.display()

# print("----------")
# new_order = roulette_selection(st_population)
# for f in (new_order):
#     f.display()

------------- Flower Details: -------------
Center Size: 19
Center Color: (31, 169, 87)
Petals Color: (76, 175, 158)
Stem Color: (199, 206, 236)
Number of Petals: 2
Fitness value: 1
array of Attributes: [19, 31, 169, 87, 76, 175, 158, 199, 206, 236, 2, 1]
------------- Flower Details: -------------
Center Size: 19
Center Color: (55, 215, 72)
Petals Color: (241, 116, 34)
Stem Color: (106, 38, 211)
Number of Petals: 6
Fitness value: 1
array of Attributes: [19, 55, 215, 72, 241, 116, 34, 106, 38, 211, 6, 1]
------------- Flower Details: -------------
Center Size: 13
Center Color: (130, 48, 109)
Petals Color: (26, 8, 141)
Stem Color: (143, 71, 123)
Number of Petals: 3
Fitness value: 1
array of Attributes: [13, 130, 48, 109, 26, 8, 141, 143, 71, 123, 3, 1]
------------- Flower Details: -------------
Center Size: 28
Center Color: (182, 33, 132)
Petals Color: (17, 196, 116)
Stem Color: (244, 190, 44)
Number of Petals: 6
Fitness value: 1
array of Attributes: [28, 182, 33, 132, 17, 196, 116, 24

# Crossover


In [158]:
def update_flower(flower1, newList):
    flower1.sizeCenter = newList[0]
    flower1.colorCenter = (newList[1] , newList[2] , newList[3])
    flower1.colorPetals = (newList[4] , newList[5] , newList[6])
    flower1.colorStem = (newList[7] , newList[8] , newList[9])
    flower1.numPetals = newList[10]
    flower1.attribute = newList



In [159]:
def crossover( flower1, flower2):
    crossPos = random.randint(1, len(flower1.attribute)-1-1)
    print(crossPos)

    # Perform crossover
    offspring1 = flower1.attribute[:crossPos] + flower2.attribute[crossPos:]
    offspring2 = flower2.attribute[:crossPos] + flower1.attribute[crossPos:]

    update_flower(flower1, offspring1)
    update_flower(flower2, offspring2)
    
    return offspring1, offspring2




In [160]:
# #test crossover
# import copy

# copy.deepcopy(population[2])  # deep copy (safer)
# flower3 = copy.deepcopy(population[1])
# flower4 = copy.deepcopy(population[2])

# flower3.display()
# flower4.display()
# off1, off2 = crossover(flower3,flower4)
# flower3.display()
# flower4.display()
# population[1].display()
# population[2].display()

# Mutation

In [161]:
def mutation(population):
    # i want for each bit to be mutated by 5%
    # mutation_rate = 0.05
    i = 0
    j = 0
    k = 0
    for flower in population:
        k += 1
        chromosome = displayBinaryRepresentation(flower)
        new_chromosome = ""
        for bit in chromosome:
            if random.random() < mut_prob:
                i = i + 1
                # Flip the bit
                if bit == '0':
                    new_bit = '1'
                else:
                    new_bit = '0'
                new_chromosome += new_bit
            else:
                new_chromosome += bit
        print("Old chromosome flower ", k, ": ", chromosome)
        print("New chromosome flower ", k, ": ", new_chromosome)
        print("Mutations in flower: ", i-j)
        j = i
        # ------------------- My code ends here this is copilot suggestion -------------------

        # Update flower attributes based on new chromosome
        flower.sizeCenter = repair(10, 30, convertBinaryToDecimal(new_chromosome[0:5]))
        flower.colorCenter = (
            repair(0, 255, convertBinaryToDecimal(new_chromosome[5:13])),
            repair(0, 255, convertBinaryToDecimal(new_chromosome[13:21])),
            repair(0, 255, convertBinaryToDecimal(new_chromosome[21:29]))
        )
        flower.colorPetals = (
            repair(0, 255, convertBinaryToDecimal(new_chromosome[29:37])),
            repair(0, 255, convertBinaryToDecimal(new_chromosome[37:45])),
            repair(0, 255, convertBinaryToDecimal(new_chromosome[45:53]))
        )
        flower.colorStem = (
            repair(0, 255, convertBinaryToDecimal(new_chromosome[53:61])),
            repair(0, 255, convertBinaryToDecimal(new_chromosome[61:69])),
            repair(0, 255, convertBinaryToDecimal(new_chromosome[69:77]))
        )
        flower.numPetals = repair(0, 7, convertBinaryToDecimal(new_chromosome[77:80]))
        flower.attribute = [flower.sizeCenter, *flower.colorCenter, *flower.colorPetals, *flower.colorStem, flower.numPetals]
    print("Total mutations: ", i)
# mutation(population)

# Evolution ( all functions working together)

In [162]:
population = createPopulation()

In [163]:
for f in population:
    print(f.attribute)

[26, 100, 31, 125, 237, 231, 163, 52, 209, 145, 6]
[11, 113, 148, 118, 58, 83, 141, 29, 111, 66, 2]
[20, 118, 120, 219, 31, 246, 148, 184, 30, 247, 5]
[25, 198, 242, 152, 113, 95, 20, 145, 198, 43, 2]
[23, 79, 123, 154, 252, 105, 243, 132, 15, 112, 5]
[19, 13, 96, 92, 161, 39, 165, 34, 210, 201, 2]
[26, 33, 168, 170, 86, 226, 45, 161, 116, 161, 5]
[18, 12, 219, 11, 65, 240, 87, 147, 21, 233, 6]


In [165]:
def evolution(population):
    # print population at the beginning
    new_population = roulette_selection(population)
    # print population after selection (ordered for crossover)
    new_population2 = copy.deepcopy(new_population)

    for i in range( 0, round(len(population)/2 * cross_prob) ):
        #print new_population2[i] and of i+1
        crossover(new_population2[i] , new_population2[i+1])
        i+=1
        print(f"crossover {i}")
        # print after crossover (just re print as crossover changes in the object) 
    
    #print population before mutation
    mutation(new_population2)
    #print populatin after mutation

    # print the population after everything 
    return new_population2


In [166]:
for f in population:
    print(f.attribute)

[26, 100, 31, 125, 237, 231, 163, 52, 209, 145, 6]
[11, 113, 148, 118, 58, 83, 141, 29, 111, 66, 2]
[20, 118, 120, 219, 31, 246, 148, 184, 30, 247, 5]
[25, 198, 242, 152, 113, 95, 20, 145, 198, 43, 2]
[23, 79, 123, 154, 252, 105, 243, 132, 15, 112, 5]
[19, 13, 96, 92, 161, 39, 165, 34, 210, 201, 2]
[26, 33, 168, 170, 86, 226, 45, 161, 116, 161, 5]
[18, 12, 219, 11, 65, 240, 87, 147, 21, 233, 6]


In [ ]:
population = evolution(population)
for f in population:
    print(f.attribute)

# GUI

In [ ]:
def rgb_str(rgb):
    return f"rgb({rgb[0]}, {rgb[1]}, {rgb[2]})"


def flower_frame_html(index, flower: Flower):
    return f"""
    <div class="flower-frame">
      <canvas id="flowerCanvas{index}" width="200" height="200"></canvas>
      <div id="timer{index}" class="timer">Timer: 0</div>
    </div>
    <script>
      (function() {{
        const canvas = document.getElementById("flowerCanvas{index}");
        const ctx = canvas.getContext("2d");
        const timerDisplay = document.getElementById("timer{index}");

        const centerSize = {flower.sizeCenter / 20:.2f};
        const centerColor = "{rgb_str(flower.colorCenter)}";
        const petalColor = "{rgb_str(flower.colorPetals)}";
        const stemColor = "{rgb_str(flower.colorStem)}";
        const numPetals = {flower.numPetals};

        function drawFlower() {{
          const centerX = 100, centerY = 100;
          const petalRadius = 15;
          const flowerRadius = 40;

          ctx.beginPath();
          ctx.strokeStyle = stemColor;
          ctx.lineWidth = 6;
          ctx.moveTo(100, 200);
          ctx.lineTo(100, 130);
          ctx.stroke();

          ctx.fillStyle = petalColor;
          for (let i = 0; i < numPetals; i++) {{
            const angle = (2 * Math.PI / numPetals) * i;
            const x = centerX + flowerRadius * Math.cos(angle);
            const y = centerY + flowerRadius * Math.sin(angle);
            ctx.beginPath();
            ctx.arc(x, y, petalRadius, 0, 2 * Math.PI);
            ctx.fill();
          }}

          ctx.beginPath();
          ctx.fillStyle = centerColor;
          ctx.arc(centerX, centerY, centerSize, 0, 2 * Math.PI);
          ctx.fill();
        }}

        drawFlower();

        let time = 0;
        let timerInterval = null;

        function startTimer() {{
          if (!timerInterval) {{
            timerInterval = setInterval(() => {{
              time += 1;
              timerDisplay.textContent = "Timer: " + time;
              window.flowerTimers[{index}] = time;
            }}, 100);
          }}
        }}

        function stopTimer() {{
          clearInterval(timerInterval);
          timerInterval = null;
        }}

        canvas.addEventListener("mouseenter", startTimer);
        canvas.addEventListener("mouseleave", stopTimer);
      }})();
    </script>
    """


def build_html(population):
    html = """
    <!doctype html>
    <html>
    <head>
      <meta charset="utf-8" />
      <style>
        body {
          font-family: Arial, sans-serif;
          display: flex;
          flex-wrap: wrap;
          justify-content: center;
          align-items: center;
          gap: 20px;
          padding: 20px;
          margin: 0;
        }
        .flower-frame {
          display: flex;
          flex-direction: column;
          align-items: center;
          justify-content: center;
          border: 2px solid #4CAF50;
          border-radius: 10px;
          background: #f9f9f9;
          width: 220px;
          height: 260px;
          box-shadow: 0 0 5px rgba(0,0,0,0.1);
          transition: box-shadow 0.3s ease;
        }
        .flower-frame:hover {
          box-shadow: 0 0 15px rgba(76, 175, 80, 0.6);
        }
        canvas {
          border-radius: 10px;
          background: #f9f9f9;
          cursor: pointer;
        }
        .timer {
          margin-top: 8px;
          font-size: 16px;
          font-weight: bold;
          color: #333;
        }
      </style>
    </head>
    <body>
    <script>window.flowerTimers = new Array(8).fill(0);</script>
    """
    for i, f in enumerate(population):
        html += flower_frame_html(i, f)
    html += "</body></html>"
    return html


# ---- Python-side logic ----
def get_best_three(timers):
    timers = [int(t) for t in timers]
    sorted_indices = sorted(range(len(timers)), key=lambda i: timers[i], reverse=True)
    top4 = sorted_indices[:4]

    result = "🏅 **Top 4 Flowers:**\n"
    medals = ["🥇", "🥈", "🥉", "🏅"]
    for rank, idx in enumerate(top4):
        result += f"{medals[rank]} Flower {idx+1} — Timer: {timers[idx]}\n"

    return result


# ---- Gradio UI ----
with gr.Blocks(theme="default") as demo:
    gr.Markdown("## 🌸 Hover over each flower to increase its timer")
    iframe_html = f'<iframe id="flowerFrame" srcdoc="{build_html(population).replace(chr(34), "&quot;")}" style="width:100%; height:850px; border:none;"></iframe>'
    gr.HTML(value=iframe_html)

    get_timers_btn = gr.Button("Show Top 3 Hovered Flowers")
    result_display = gr.Markdown("")

    get_timers_btn.click(
        fn=lambda timers: get_best_three(timers),
        js="window.flowerTimers",
        outputs=result_display,
    )

demo.launch()

c:\Users\SHANA\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\utils.py:1052: UserWarning: Expected 1 arguments for function <function <lambda> at 0x000002111CB8E660>, received 0.
  warnings.warn(
c:\Users\SHANA\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\utils.py:1056: UserWarning: Expected at least 1 arguments for function <function <lambda> at 0x000002111CB8E660>, received 0.
  warnings.warn(


In [35]:
demo.close()
